<a href="https://colab.research.google.com/github/mysticizhere/ML/blob/main/Tensorflow_HiddenMarkov_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!pip install tensorflow_probability==0.8.0rc0 --user --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 10.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.25.2 requires cloudpickle>=1.2.0, but you have cloudpickle 1.1.1 which is incompatible.
distributed 2022.2.0 requires cloudpickle>=1.5.0, but you have cloudpickle 1.1.1 which is incompatible.


In [3]:
import tensorflow_probability as tfp  # We are using a different module from tensorflow this time
import tensorflow as tf

**Weather Model**

We will model a simple weather system and try to predict the temperature on each day given the following information.

*   Cold days are encoded by a 0 and hot days are encoded by a 1.
*   The first day in our sequence has an 80% chance of being cold.
*   A cold day has a 30% chance of being followed by a hot day.
*   A hot day has a 20% chance of being followed by a cold day.
*   On each day the temperature is normally distributed with mean and standard deviation 0 and 5 on a cold day and mean and standard deviation 15 and 10 on a hot day.






In [31]:
tfd = tfp.distributions  # making a shortcut for later on
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])  # Refer to point 2 above
transition_distribution = tfd.Categorical(probs=[[0.9, 0.1],
                                                 [0.5, 0.5]])  # refer to points 3 and 4 above
observation_distribution = tfd.Normal(loc=[0., 30.], scale=[5., 15.])  # refer to point 5 above

# the loc argument represents the mean and the scale is the standard devitation

We've now created distribution variables to model our system and it's time to create the hidden markov model.

In [32]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=335)
#num_steps is the number of times the model will predict the values for, 
#for this example its the number of days for which it will predict the temperature for

Lets see the output !

In [33]:
mean = model.mean()

# due to the way TensorFlow works on a lower level we need to evaluate part of the graph
# from within a session to see the value of this tensor

# in the new version of tensorflow we need to use tf.compat.v1.Session() rather than just tf.Session()
with tf.compat.v1.Session() as sess:  
  print(mean.numpy())

[5.9999995 5.3999996 5.16      5.064     5.0255985 5.0102386 5.0040956
 5.0016375 5.0006547 5.0002604 5.0001025 5.000039  5.000014  5.0000043
 4.9999995 4.999998  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.999997  4.999997  4.999997  4.999997  4.999997  4.999997
 4.999997  4.99999

The above are the expected temperatures for next seven days !